# Testing of ipyleaflet

Alternative for maping is Folium

Installed for:
- python>=3.6
- primarily relying on conda install
- ipyleaflet was installed via pip as conda broke with numpy exceptions
- install nodejs via conda
- all other install steps were followed via the [install instructions](https://ipyleaflet.readthedocs.io/en/latest/installation.html#using-conda)
- multiple conda kernels are on the development machine so ipykernels needs to be installed too

In [11]:
import warnings
warnings.filterwarnings(action='ignore', message="numpy.dtype size changed,")

from ipyleaflet import Map, basemaps, basemap_to_tiles, Circle

import pandas as pd
import cmocean
import numpy as np

import datetime
import time

## Setup the view

The user will need to set the following parameters

`network = True` The base image is drawn from MODIS retrievals from the most recent day - internet must be available

`center=(lat,lon)` - (latN,lonE)

`zoom=4` level of zoomed in or out 1(far)-9(close)

In [12]:
#setup basemap and view that will be updated
def set_basemap():
    network=True
    center=(65, 200)
    zoom=4

    if network:
        m = Map(
                layers=(basemap_to_tiles(basemaps.NASAGIBS.ModisTerraTrueColorCR, 
                                         (datetime.datetime.today()-datetime.timedelta(1)).strftime('%Y-%m-%d')), ),
            center=center,
            zoom=zoom
        )
    else:
        m = Map(
            center=center,
            zoom=zoom
        )
    return m

In [13]:
image_date=(datetime.datetime.today()-datetime.timedelta(1))

print(f'The base image is from: {image_date:%B %d, %Y}')

The base image is from: August 21, 2018


### Adding Data Locations Color-Coded by value

We add a circle where the inner and outer ring are coded by color and location.  Currently the size of the circle is fixed, but this could present another variable.  

In [14]:
def add_circle(lat,lon,value,color):
    circle.location = (lat, 360+lon)
    circle.radius = 500
    circle.value = value
    circle.color = color
    circle.fill_color = color
    
    return circle


Since the leaflet routines take hex values, we have to convert the cmocean scale to hex (since i like this scale)

In [15]:
#use cmocean colormaps
def cmocean_to_hex(cmap, pl_entries):
    h = 1.0/(pl_entries-1)
    pl_colorscale = []

    for k in range(pl_entries):
        C = list(map(np.uint8, np.array(cmap(k*h)[:3])*255))
        pl_colorscale.append([k*h, '#{:02x}{:02x}{:02x}'.format(C[0],C[1],C[2])],)

    return pl_colorscale

cm_thermal = cmocean_to_hex(cmocean.cm.thermal,256)
cm_val = [a for a,b in iter(cm_thermal)]
cm_hex = [b for a,b in iter(cm_thermal)]

## Read in IWG data file to be parsed/plotted

In [16]:
iwg_file="C:\\Users\\pmelctd\\Documents\\2018 Downloads\\20180525_221008_IWG.clean.csv"

#### Normalize SST to min and max values specified below

#### Downsample the input data for speed and clarity

resample values are strings eg:
- '30s': 30 seconds
- '1t': 1 minute
- '90s': 90 seconds

In [17]:
def load_data():
    df = pd.read_csv(iwg_file,
                     parse_dates=['TIME'],
                     index_col='TIME')

    #set max and min ranges to scale to
    maxval=10
    minval=-2

    df['norm'] = df['SST'].apply(lambda x: (x - (minval)) / (maxval - (-2)))

    #set values outside of max/min to be the same as max/min
    df['norm'][df['norm']>=1]=1
    df['norm'][df['norm']<0]=0

    df_downsample = df.resample('60s').median()
    
    return df_downsample

In [18]:
m = set_basemap()
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [20]:
df_downsample=load_data()

for count in range(0,int(len(df_downsample)/1),1):
    circle = Circle()

    circle = add_circle(df_downsample['LAT'][count],
                        df_downsample['LON'][count],
                        df_downsample['norm'][count],
                        color=cm_hex[np.searchsorted(cm_val, df_downsample['norm'][count], side="left")])
    m.add_layer(circle)
    count+=1




C:\Users\pmelctd\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\Users\pmelctd\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
